In [ ]:
import pyelegant as pe
from pyelegant import elebuilder
from pyelegant import remote
from pyelegant import sdds
from pyelegant import util
from pyelegant import notation

In [ ]:
ed = elebuilder.EleDesigner()

rf = ed.rpnfuncs
rd = ed.rpnvars['optimization_term'].dict

ini_LTE_filepath = 'nsls2cb72pm_LS_SS_LS_ZeroSexts.lte'
use_beamline = 'CELL_LS_SS_LS'

nsls2_flr_filepath = '/GPFS/APC/yhidaka/git_repos/nsls2cb/nsls2.flr'

ed.add_block('run_setup',
    lattice = ini_LTE_filepath,
    p_central_mev = 3e3, use_beamline=use_beamline, semaphore_file = '%s.done',
    parameters='%s.param', default_order = 2)

ed.add_newline()
ed.add_block('run_control')

ed.add_newline()
ed.add_comment('! Use simplex optimization')
ed.add_block('optimization_setup', mode = 'minimize', method = 'simplex')

ed.add_newline()
ed.add_comment('! Dipole Angles')
for name in ['BD', 'BF', 'LBD', 'LBF', 'HBD', 'HBF', 'MBD', 'MBF']:
    ed.add_block('optimization_variable',
        name = name, item = 'ANGLE',
        lower_limit=-0.01, upper_limit=0.03, step_size=1e-5)

eq = rf.dtor(24) - (
    rd['BD.ANGLE'] * ed.get_LTE_elem_count('BD') + 
    rd['BF.ANGLE'] * ed.get_LTE_elem_count('BF') +
    rd['LBD.ANGLE'] * ed.get_LTE_elem_count('LBD') +
    rd['LBF.ANGLE'] * ed.get_LTE_elem_count('LBF') +
    rd['HBD.ANGLE'] * ed.get_LTE_elem_count('HBD') +
    rd['HBF.ANGLE'] * ed.get_LTE_elem_count('HBF') +
    rd['MBD.ANGLE'] * ed.get_LTE_elem_count('MBD') +
    rd['MBF.ANGLE'] * ed.get_LTE_elem_count('MBF')
)
ed.add_block('optimization_term',term = rf.abs(eq))

ed.add_block('optimization_term',term = "Particles 1 1e-10 sene")
#ed.add_block('optimization_term',term = "M_SS#1.Particles 1 1e-10 sene")
#ed.add_block('optimization_term',term = "Particles")

ed.add_block('bunched_beam')

ed.add_comment("! Start optimization")
ed.add_block('optimize')

ed.write()